In [3]:
import os 
import glob
from pathlib import Path
import pandas as pd 
import numpy as np 
import nibabel as nib 
import json
from nilearn import plotting, image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

/working/lupolab/julia/anaconda/envs/pytnew/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


# 1 Map the segmentations (named for T1c image) to their corresponding patient: 

Begin by reading in the json file and data frames that describe what we just segmented: 

In [4]:
PATH = Path('/working/lupolab/julia/tcia_analysis/datasets/')

In [5]:
df = pd.read_csv(Path(PATH, 'one_exam_per_patient_with_fixed_vol.csv'))

In [6]:
with open(Path(PATH.parent,'nvidia', 'config', 'config_datalist_t1c.json'), 'r') as f: 
    segment_json = json.load(f)

In [7]:
segment_json

{'training': [],
 'validation': [{'image': 'TCGA-GBM-nifti/TCGA-06-0240/07-02-2005-04728/9-AxT1-thin_for_surgery-27598a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0143/02-11-2005-83875/11-COR__T1_POST_GD_FLAIR-28783a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0149/03-25-2003-87536/10-COR_T1_POST_GD_FLAIR-88922a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-19-0955/04-07-2001-NR_MRI_BRAIN_WWO-83151/8-MRHG_T1_AX_POST_GAD-01973a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0648/01-20-2006-51419/11-AX_T1_POST_GD_FLAIR-02197a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-12-0829/06-02-1999-BRAIN-09460/18-T1_AX_POST-73832a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0171/08-04-2001-68007/9-AX_T1_POST_GD_FLAIR-32350a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-14-1401/04-18-1997-MRI_BRAIN_W_WOUT_CONTRAST-27628/1201-BRAIN_WWO_T1_POST-04167a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-02-0033/05-26-1997-MRI_BRAIN_WWO_CONTR-89744/6-AX_T1_POST-51465a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-19

Now list all of the segmentations that are in our 'eval_t1c' folder: 

In [8]:
segmentations = [Path(x).stem for x in glob.glob(str(Path(PATH.parent, 'nvidia', 'eval_t1c', '*a'))) if x not in glob.glob(str(Path(PATH.parent, 'nvidia', 'eval_t1c', '*t1ca')))]

In [9]:
len(segmentations)

226

In [10]:
segmentations[:10]

['9-AxT1-thin_for_surgery-27598a',
 '11-COR__T1_POST_GD_FLAIR-28783a',
 '10-COR_T1_POST_GD_FLAIR-88922a',
 '8-MRHG_T1_AX_POST_GAD-01973a',
 '11-AX_T1_POST_GD_FLAIR-02197a',
 '18-T1_AX_POST-73832a',
 '9-AX_T1_POST_GD_FLAIR-32350a',
 '1201-BRAIN_WWO_T1_POST-04167a',
 '6-AX_T1_POST-51465a',
 '8-MP_RAGE_AXIAL-23748a']

In [11]:
df.head()

,patient_id,cohort,patient_exam,image,contrast,contrast_label,plane,exam_eligible,fixed_vol
0,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,9-AxT1-thin_for_surgery-27598,1,T1C,ax,1,4-AxFLAIR-thin_for_surgery-47036
1,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,21-nordicICE_HFH_-_rBF_map_-Leakage_corrected-...,1,rBF,cor,1,4-AxFLAIR-thin_for_surgery-47036
2,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,1-3_PLANE_LOC-22369,0,LOCALIZER,NaN,1,4-AxFLAIR-thin_for_surgery-47036
3,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,5-AXIAL_DIFFUSION-62807,0,DWI,ax,1,4-AxFLAIR-thin_for_surgery-47036
4,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,8-AXIAL_PERFUSION-44568,1,PERFUSION,ax,1,4-AxFLAIR-thin_for_surgery-47036


In [12]:
df.shape

(3630, 9)

In [13]:
df_nodup = df.drop_duplicates(subset=['patient_id'])
df_nodup = df_nodup.drop(['image', 'contrast', 'contrast_label', 'plane', 'exam_eligible', 'fixed_vol'], axis = 1)

In [14]:
df_nodup.shape

(274, 3)

In [15]:
df_nodup = df_nodup.reset_index(drop = True)

In [16]:
df_nodup.head()

,patient_id,cohort,patient_exam
0,TCGA-06-0240,TCGA-GBM,07-02-2005-04728
1,TCGA-06-0143,TCGA-GBM,02-11-2005-83875
2,TCGA-06-0149,TCGA-GBM,03-25-2003-87536
3,TCGA-19-0955,TCGA-GBM,04-07-2001-NR_MRI_BRAIN_WWO-83151
4,TCGA-06-0648,TCGA-GBM,01-20-2006-51419


In [17]:
seg_loc_root = Path(PATH.parent, 'nvidia', 'eval_t1c')

In [18]:
seg_list = [x['image'] for x in segment_json['validation']]

In [19]:
seg_list

['TCGA-GBM-nifti/TCGA-06-0240/07-02-2005-04728/9-AxT1-thin_for_surgery-27598a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0143/02-11-2005-83875/11-COR__T1_POST_GD_FLAIR-28783a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0149/03-25-2003-87536/10-COR_T1_POST_GD_FLAIR-88922a.nii.gz',
 'TCGA-GBM-nifti/TCGA-19-0955/04-07-2001-NR_MRI_BRAIN_WWO-83151/8-MRHG_T1_AX_POST_GAD-01973a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0648/01-20-2006-51419/11-AX_T1_POST_GD_FLAIR-02197a.nii.gz',
 'TCGA-GBM-nifti/TCGA-12-0829/06-02-1999-BRAIN-09460/18-T1_AX_POST-73832a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0171/08-04-2001-68007/9-AX_T1_POST_GD_FLAIR-32350a.nii.gz',
 'TCGA-GBM-nifti/TCGA-14-1401/04-18-1997-MRI_BRAIN_W_WOUT_CONTRAST-27628/1201-BRAIN_WWO_T1_POST-04167a.nii.gz',
 'TCGA-GBM-nifti/TCGA-02-0033/05-26-1997-MRI_BRAIN_WWO_CONTR-89744/6-AX_T1_POST-51465a.nii.gz',
 'TCGA-GBM-nifti/TCGA-19-1390/03-20-2001-NR_MRI_BRAIN_WWO-90326/8-MP_RAGE_AXIAL-23748a.nii.gz',
 'TCGA-GBM-nifti/TCGA-08-0356/03-26-2002-MSTEALTH-31923/7-AX_3D_SPGR_GAD-17095

In [20]:
attempted_segmentations = pd.DataFrame({'patient_id': [x.split('/')[1] for x in seg_list], 
                                        'cohort': [x.split('/')[0] for x in seg_list], 
                                        'patient_exam': [x.split('/')[2] for x in seg_list], 
                                        
             'seg_loc': [x.split('/')[3].split('.')[-3] for x in seg_list], 
             'seg_file': [ x.split('/')[3].split('.')[-3]+'_TC.nii.gz' for x in seg_list]})

In [21]:
actual_segmentations = pd.DataFrame(segmentations)

In [22]:
actual_segmentations.shape, attempted_segmentations.shape

((226, 1), (231, 5))

In [23]:
actual_segmentations.head()

,0
0,9-AxT1-thin_for_surgery-27598a
1,11-COR__T1_POST_GD_FLAIR-28783a
2,10-COR_T1_POST_GD_FLAIR-88922a
3,8-MRHG_T1_AX_POST_GAD-01973a
4,11-AX_T1_POST_GD_FLAIR-02197a


In [24]:
attempted_segmentations.head()

,patient_id,cohort,patient_exam,seg_loc,seg_file
0,TCGA-06-0240,TCGA-GBM-nifti,07-02-2005-04728,9-AxT1-thin_for_surgery-27598a,9-AxT1-thin_for_surgery-27598a_TC.nii.gz
1,TCGA-06-0143,TCGA-GBM-nifti,02-11-2005-83875,11-COR__T1_POST_GD_FLAIR-28783a,11-COR__T1_POST_GD_FLAIR-28783a_TC.nii.gz
2,TCGA-06-0149,TCGA-GBM-nifti,03-25-2003-87536,10-COR_T1_POST_GD_FLAIR-88922a,10-COR_T1_POST_GD_FLAIR-88922a_TC.nii.gz
3,TCGA-19-0955,TCGA-GBM-nifti,04-07-2001-NR_MRI_BRAIN_WWO-83151,8-MRHG_T1_AX_POST_GAD-01973a,8-MRHG_T1_AX_POST_GAD-01973a_TC.nii.gz
4,TCGA-06-0648,TCGA-GBM-nifti,01-20-2006-51419,11-AX_T1_POST_GD_FLAIR-02197a,11-AX_T1_POST_GD_FLAIR-02197a_TC.nii.gz


In [25]:
actual_seg_map = attempted_segmentations.loc[attempted_segmentations.seg_loc.isin(list(actual_segmentations[0]))]

In [26]:
actual_seg_map.shape

(226, 5)

In [27]:
actual_seg_map['t1c'] = [x[:-10]+'.nii.gz' for x in actual_seg_map.seg_file]

/working/lupolab/julia/anaconda/envs/pytnew/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
actual_seg_map.head()

,patient_id,cohort,patient_exam,seg_loc,seg_file,t1c
0,TCGA-06-0240,TCGA-GBM-nifti,07-02-2005-04728,9-AxT1-thin_for_surgery-27598a,9-AxT1-thin_for_surgery-27598a_TC.nii.gz,9-AxT1-thin_for_surgery-27598a.nii.gz
1,TCGA-06-0143,TCGA-GBM-nifti,02-11-2005-83875,11-COR__T1_POST_GD_FLAIR-28783a,11-COR__T1_POST_GD_FLAIR-28783a_TC.nii.gz,11-COR__T1_POST_GD_FLAIR-28783a.nii.gz
2,TCGA-06-0149,TCGA-GBM-nifti,03-25-2003-87536,10-COR_T1_POST_GD_FLAIR-88922a,10-COR_T1_POST_GD_FLAIR-88922a_TC.nii.gz,10-COR_T1_POST_GD_FLAIR-88922a.nii.gz
3,TCGA-19-0955,TCGA-GBM-nifti,04-07-2001-NR_MRI_BRAIN_WWO-83151,8-MRHG_T1_AX_POST_GAD-01973a,8-MRHG_T1_AX_POST_GAD-01973a_TC.nii.gz,8-MRHG_T1_AX_POST_GAD-01973a.nii.gz
4,TCGA-06-0648,TCGA-GBM-nifti,01-20-2006-51419,11-AX_T1_POST_GD_FLAIR-02197a,11-AX_T1_POST_GD_FLAIR-02197a_TC.nii.gz,11-AX_T1_POST_GD_FLAIR-02197a.nii.gz


In [29]:
tcia_df = actual_seg_map

# 2 Shift the segmentations to match the original header 

In [30]:
base_dest_path = '/working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/'

In [31]:
PATH_TCIA = Path("/working/lupolab/julia/tcia_analysis")

Find the original paths for the images: 

In [32]:
t1c_file_paths = []
for idx, row in tcia_df.iterrows():
    if type(row['t1c']) != float: 
        orig_t1c_file_path = str(Path(PATH_TCIA, 'datasets',row['cohort'], row['patient_id'], row['patient_exam'],  row['t1c']))
        t1c_file_paths.append(orig_t1c_file_path)

In [33]:
os.chdir(base_dest_path)

In [34]:
# !rm -r *_t1ca

rm: cannot remove ‘*_t1ca’: No such file or directory


Read in the original image and the segmentation, shift the segmentation & save it in the same folder as the segmentation: 

In [35]:
from nilearn.image import new_img_like

In [36]:
for idx, row in actual_seg_map.iterrows():
    original_file     = Path(PATH_TCIA, 'datasets', row['cohort'], row['patient_id'], row['patient_exam'], row['t1c'])
    segmentation_file = Path(PATH_TCIA, 'nvidia', 'eval_t1c', row['seg_loc'], row['seg_file']) 
    orig_t1c_nii      = nib.load(original_file)
    seg_t1c_nii       = nib.load(segmentation_file)
    shifted_nifti     = new_img_like(orig_t1c_nii, 
                                     seg_t1c_nii.get_fdata(), 
                                     affine=orig_t1c_nii.affine, 
                                     copy_header=True)
    # Ensuring that the shapes match: 
    if shifted_nifti.shape == orig_t1c_nii.shape:
        print(f"Shapes match for {row['patient_id']}")
    else: print(f"Shapes don't match for {row['patient_id']}") 
    
    # Save the shifted nifti in the folder where the original segmentation was: 
    shifted_seg_name = Path(segmentation_file.parent, row['seg_file'][:-7]+'_shifted'+row['seg_file'][-7:])
    print(f'Saving {shifted_seg_name}')
    nib.save(shifted_nifti, Path(segmentation_file.parent, shifted_seg_name))

Shapes match for TCGA-06-0240
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/9-AxT1-thin_for_surgery-27598a/9-AxT1-thin_for_surgery-27598a_TC_shifted.nii.gz
Shapes match for TCGA-06-0143
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/11-COR__T1_POST_GD_FLAIR-28783a/11-COR__T1_POST_GD_FLAIR-28783a_TC_shifted.nii.gz
Shapes match for TCGA-06-0149
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/10-COR_T1_POST_GD_FLAIR-88922a/10-COR_T1_POST_GD_FLAIR-88922a_TC_shifted.nii.gz
Shapes match for TCGA-19-0955
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/8-MRHG_T1_AX_POST_GAD-01973a/8-MRHG_T1_AX_POST_GAD-01973a_TC_shifted.nii.gz
Shapes match for TCGA-06-0648
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/11-AX_T1_POST_GD_FLAIR-02197a/11-AX_T1_POST_GD_FLAIR-02197a_TC_shifted.nii.gz
Shapes match for TCGA-12-0829
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/18-T1_AX_POST-73832a/18-T1_AX_POST-73832a_TC_shifted.nii.gz
Shap

Shapes match for TCGA-08-0358
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/8-T1_AX_POST_GAD-12257a/8-T1_AX_POST_GAD-12257a_TC_shifted.nii.gz
Shapes match for TCGA-02-0102
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/8-SAG_T1_POST-42151a/8-SAG_T1_POST-42151a_TC_shifted.nii.gz
Shapes match for TCGA-06-0166
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/7-AXIAL_T1_POST_GD-25391a/7-AXIAL_T1_POST_GD-25391a_TC_shifted.nii.gz
Shapes match for TCGA-06-0188
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/9-COR_T1_GD-04859a/9-COR_T1_GD-04859a_TC_shifted.nii.gz
Shapes match for TCGA-06-0182
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/13-AxT1-thin_for_surgery-12089a/13-AxT1-thin_for_surgery-12089a_TC_shifted.nii.gz
Shapes match for TCGA-02-0037
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/8-SAG_T1_POST-08358a/8-SAG_T1_POST-08358a_TC_shifted.nii.gz
Shapes match for TCGA-08-0360
Saving /working/lupolab/julia/tcia_a

Shapes match for TCGA-12-1094
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/7-POST_AX_T1-19749a/7-POST_AX_T1-19749a_TC_shifted.nii.gz
Shapes match for TCGA-19-5950
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/6-MP_RAGE_AXIAL-14826a/6-MP_RAGE_AXIAL-14826a_TC_shifted.nii.gz
Shapes match for TCGA-06-0187
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/12-AxT1-thin_for_surgery-42560a/12-AxT1-thin_for_surgery-42560a_TC_shifted.nii.gz
Shapes match for TCGA-06-0241
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/11-COR__T1_POST_GD_FLAIR-59018a/11-COR__T1_POST_GD_FLAIR-59018a_TC_shifted.nii.gz
Shapes match for TCGA-08-0392
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/7-3D_SPGR_AX-62668a/7-3D_SPGR_AX-62668a_TC_shifted.nii.gz
Shapes match for TCGA-27-2524
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/5-t1setra-74870a/5-t1setra-74870a_TC_shifted.nii.gz
Shapes match for TCGA-14-0789
Saving /working/lupolab/julia/tcia

Shapes match for TCGA-DU-8163
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/13-COR__T1_POST_GD_FLAIR-85227a/13-COR__T1_POST_GD_FLAIR-85227a_TC_shifted.nii.gz
Shapes match for TCGA-DU-A5TU
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/15-Axial_T1_FSE_Post_Gad-17816a/15-Axial_T1_FSE_Post_Gad-17816a_TC_shifted.nii.gz
Shapes match for TCGA-DU-7018
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/14-COR__T1_POST_GD_FLAIR-46276a/14-COR__T1_POST_GD_FLAIR-46276a_TC_shifted.nii.gz
Shapes match for TCGA-HT-7881
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/6-CAX_3D_SPGR_POST-39088a/6-CAX_3D_SPGR_POST-39088a_TC_shifted.nii.gz
Shapes match for TCGA-DU-5872
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/14-COR_T1_POST_GD-35911a/14-COR_T1_POST_GD-35911a_TC_shifted.nii.gz
Shapes match for TCGA-DU-6404
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/10-AXIAL_T1_POST_GD-43862a/10-AXIAL_T1_POST_GD-43862a_TC_shifted.nii.gz
Shap

Shapes match for TCGA-FG-A87N
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/11-t1mpragetra_Gd-25909a/11-t1mpragetra_Gd-25909a_TC_shifted.nii.gz
Shapes match for TCGA-DU-6397
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/9-AXIAL_T1_POST_GD-37336a/9-AXIAL_T1_POST_GD-37336a_TC_shifted.nii.gz
Shapes match for TCGA-HT-7605
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/20-C_Ax_T1_MP_SPGR-72811a/20-C_Ax_T1_MP_SPGR-72811a_TC_shifted.nii.gz
Shapes match for TCGA-HT-8105
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/12-SAG_T1C-10889a/12-SAG_T1C-10889a_TC_shifted.nii.gz
Shapes match for TCGA-HT-7602
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/11-SAG_T1C-55726a/11-SAG_T1C-55726a_TC_shifted.nii.gz
Shapes match for TCGA-HT-7608
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_t1c/12-Ax_T1_MP_SPGR__GAD-05970a/12-Ax_T1_MP_SPGR__GAD-05970a_TC_shifted.nii.gz
Shapes match for TCGA-CS-6290
Saving /working/lupolab/julia/tcia_ana

## 2.2 Check to make sure that all the segmentations have a corresponding shifted segmentation: 

In [39]:
segmentations = glob.glob(base_dest_path+'/*/*a_TC.nii.gz')
len(segmentations)

226

In [40]:
segmentations = glob.glob(base_dest_path+'/*/*a_TC_shifted.nii.gz')
len(segmentations)

226

## 2.3 Change the data frame to point to the shifted segmentations instead of the original ones: 

In [41]:
actual_seg_map['seg_file_shifted'] = [x[:-7]+'_shifted'+x[-7:] for x in actual_seg_map.seg_file]

/working/lupolab/julia/anaconda/envs/pytnew/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
actual_seg_map = actual_seg_map.drop(['seg_file'], axis = 1)


In [43]:
actual_seg_map.head()

,patient_id,cohort,patient_exam,seg_loc,t1c,seg_file_shifted
0,TCGA-06-0240,TCGA-GBM-nifti,07-02-2005-04728,9-AxT1-thin_for_surgery-27598a,9-AxT1-thin_for_surgery-27598a.nii.gz,9-AxT1-thin_for_surgery-27598a_TC_shifted.nii.gz
1,TCGA-06-0143,TCGA-GBM-nifti,02-11-2005-83875,11-COR__T1_POST_GD_FLAIR-28783a,11-COR__T1_POST_GD_FLAIR-28783a.nii.gz,11-COR__T1_POST_GD_FLAIR-28783a_TC_shifted.nii.gz
2,TCGA-06-0149,TCGA-GBM-nifti,03-25-2003-87536,10-COR_T1_POST_GD_FLAIR-88922a,10-COR_T1_POST_GD_FLAIR-88922a.nii.gz,10-COR_T1_POST_GD_FLAIR-88922a_TC_shifted.nii.gz
3,TCGA-19-0955,TCGA-GBM-nifti,04-07-2001-NR_MRI_BRAIN_WWO-83151,8-MRHG_T1_AX_POST_GAD-01973a,8-MRHG_T1_AX_POST_GAD-01973a.nii.gz,8-MRHG_T1_AX_POST_GAD-01973a_TC_shifted.nii.gz
4,TCGA-06-0648,TCGA-GBM-nifti,01-20-2006-51419,11-AX_T1_POST_GD_FLAIR-02197a,11-AX_T1_POST_GD_FLAIR-02197a.nii.gz,11-AX_T1_POST_GD_FLAIR-02197a_TC_shifted.nii.gz


In [44]:
actual_seg_map.columns = ['patient_id', 'cohort', 'patient_exam', 'seg_loc', 't1c', 'seg_file']

In [45]:
seg_df = actual_seg_map

In [46]:
seg_df['seg_loc'] = [str(Path(seg_loc_root, x)) for x in list(seg_df['seg_loc'])]

In [47]:
seg_df.head()

,patient_id,cohort,patient_exam,seg_loc,t1c,seg_file
0,TCGA-06-0240,TCGA-GBM-nifti,07-02-2005-04728,/working/lupolab/julia/tcia_analysis/nvidia/ev...,9-AxT1-thin_for_surgery-27598a.nii.gz,9-AxT1-thin_for_surgery-27598a_TC_shifted.nii.gz
1,TCGA-06-0143,TCGA-GBM-nifti,02-11-2005-83875,/working/lupolab/julia/tcia_analysis/nvidia/ev...,11-COR__T1_POST_GD_FLAIR-28783a.nii.gz,11-COR__T1_POST_GD_FLAIR-28783a_TC_shifted.nii.gz
2,TCGA-06-0149,TCGA-GBM-nifti,03-25-2003-87536,/working/lupolab/julia/tcia_analysis/nvidia/ev...,10-COR_T1_POST_GD_FLAIR-88922a.nii.gz,10-COR_T1_POST_GD_FLAIR-88922a_TC_shifted.nii.gz
3,TCGA-19-0955,TCGA-GBM-nifti,04-07-2001-NR_MRI_BRAIN_WWO-83151,/working/lupolab/julia/tcia_analysis/nvidia/ev...,8-MRHG_T1_AX_POST_GAD-01973a.nii.gz,8-MRHG_T1_AX_POST_GAD-01973a_TC_shifted.nii.gz
4,TCGA-06-0648,TCGA-GBM-nifti,01-20-2006-51419,/working/lupolab/julia/tcia_analysis/nvidia/ev...,11-AX_T1_POST_GD_FLAIR-02197a.nii.gz,11-AX_T1_POST_GD_FLAIR-02197a_TC_shifted.nii.gz


In [48]:
seg_df.to_csv(Path(PATH, 'segmentation_t1c_df.csv'), index = False)

In [49]:
os.listdir(PATH)

['.ipynb_checkpoints',
 'TCGA-GBM-nifti',
 'TCGA-LGG-nifti',
 'come_back_to',
 'manifest_files',
 'dicom_images_with_contrast.csv',
 'contrast_labels.csv',
 'contrast_labels_with_flair_eligibility.csv',
 'contrast_labels_with_eligibility.csv',
 'images_with_dicom_contrast.csv',
 'exam_counts.csv',
 'exam_counts.txt',
 'one_exam_per_patient_with_labels_flair.csv',
 'one_exam_per_patient_with_fixed_vol.csv',
 'one_exam_per_patient_with_labels.csv',
 'rename_script.sh',
 'segmentation_df.csv',
 'tcga_subtype_data.csv',
 'segmentation_fla_df.csv',
 'segmentation_t1c_df.csv']

# 3 Write out the pngs to ensure that they look ok: 

In [50]:
## Let's investigate just a few of them to ensure that they are correct: 
import nibabel as nib
import matplotlib.pyplot as plt

In [51]:
%matplotlib inline

In [52]:
plotting_dict = {'display_mode':'z', 
                 'cut_coords':[-50,-30,-10, 10, 30, 50], 
                 'cmap':cm.get_cmap('viridis')}

In [ ]:
for idx, row in seg_df.iterrows(): 
    
    original_file     = Path(PATH_TCIA, 'datasets', row['cohort'], row['patient_id'], row['patient_exam'], row['t1c'])
    segmentation_file = Path(PATH_TCIA, 'nvidia', 'eval_t1c', row['seg_loc'], row['seg_file']) 
    orig_t1c_nii      = nib.load(original_file)
    seg_t1c_nii       = nib.load(segmentation_file)
    
    png_path = Path(base_dest_path, 'png_output')
    os.makedirs(png_path, exist_ok = True)
    print(f'Saving images for '+str(row['patient_id']))
    plotting.plot_img(orig_t1c_nii, title='T1C', output_file=str(Path(png_path, str(row['patient_id'])+'t1ca.png')), **plotting_dict)
    plotting.plot_stat_map(seg_t1c_nii, orig_t1c_nii, output_file=str(Path(png_path, str(row['patient_id'])+'seg.png')), title='Segmentation', **plotting_dict)
    

Saving images for TCGA-06-0240
Saving images for TCGA-06-0143
Saving images for TCGA-06-0149
Saving images for TCGA-19-0955
Saving images for TCGA-06-0648
Saving images for TCGA-12-0829
Saving images for TCGA-06-0171
Saving images for TCGA-14-1401
Saving images for TCGA-02-0033
Saving images for TCGA-19-1390
Saving images for TCGA-08-0356
Saving images for TCGA-02-0106
Saving images for TCGA-06-0878
Saving images for TCGA-06-0168
Saving images for TCGA-06-0162
Saving images for TCGA-76-6193
Saving images for TCGA-06-0165
Saving images for TCGA-14-0813
Saving images for TCGA-12-1098
Saving images for TCGA-02-0006


/home/sf673542/.local/lib/python3.7/site-packages/nilearn/plotting/displays.py:780: UserWarning: empty mask
  get_mask_bounds(new_img_like(img, not_mask, affine))


Saving images for TCGA-08-0520
Saving images for TCGA-02-0034
Saving images for TCGA-14-1821
Saving images for TCGA-06-0176
Saving images for TCGA-02-0068
Saving images for TCGA-02-0086
Saving images for TCGA-06-5413
Saving images for TCGA-06-0133
Saving images for TCGA-06-0139
Saving images for TCGA-02-0027
Saving images for TCGA-06-1802
Saving images for TCGA-06-0137
Saving images for TCGA-06-5417
Saving images for TCGA-06-1806
Saving images for TCGA-76-4935
Saving images for TCGA-08-0390
Saving images for TCGA-02-0054
Saving images for TCGA-06-0178
Saving images for TCGA-14-1402
Saving images for TCGA-14-1825
Saving images for TCGA-02-0116
Saving images for TCGA-02-0011
Saving images for TCGA-08-0524
Saving images for TCGA-14-1454
Saving images for TCGA-02-0047
Saving images for TCGA-76-4926
Saving images for TCGA-08-0389
Saving images for TCGA-19-5960
Saving images for TCGA-06-0185
Saving images for TCGA-02-0075
Saving images for TCGA-14-0817
Saving images for TCGA-08-0355
Saving i

/home/sf673542/.local/lib/python3.7/site-packages/nilearn/plotting/displays.py:780: UserWarning: empty mask
  get_mask_bounds(new_img_like(img, not_mask, affine))


Saving images for TCGA-06-0175
Saving images for TCGA-02-0059
Saving images for TCGA-06-1801
Saving images for TCGA-06-2570
Saving images for TCGA-76-4932
Saving images for TCGA-19-0963
Saving images for TCGA-06-0147
Saving images for TCGA-06-0130
Saving images for TCGA-14-1037


...

Saving images for TCGA-12-0776
Saving images for TCGA-76-6656
Saving images for TCGA-02-0069
Saving images for TCGA-06-0644
Saving images for TCGA-02-0087
Saving images for TCGA-14-3477
Saving images for TCGA-76-6282
Saving images for TCGA-06-0138
Saving images for TCGA-06-5412
Saving images for TCGA-06-0145
Saving images for TCGA-06-0132
Saving images for TCGA-14-1395
Saving images for TCGA-76-6664
Saving images for TCGA-06-0164
Saving images for TCGA-06-0119
Saving images for TCGA-76-6192
Saving images for TCGA-02-0070
Saving images for TCGA-12-1093


/home/sf673542/.local/lib/python3.7/site-packages/nilearn/plotting/displays.py:780: UserWarning: empty mask
  get_mask_bounds(new_img_like(img, not_mask, affine))


Saving images for TCGA-14-0812
Saving images for TCGA-14-0865
Saving images for TCGA-06-0156
Saving images for TCGA-08-0521
Saving images for TCGA-27-1836
Saving images for TCGA-02-0048
Saving images for TCGA-14-0790
Saving images for TCGA-12-1601
Saving images for TCGA-14-1456
Saving images for TCGA-12-1094
Saving images for TCGA-19-5950


/home/sf673542/.local/lib/python3.7/site-packages/nilearn/plotting/displays.py:780: UserWarning: empty mask
  get_mask_bounds(new_img_like(img, not_mask, affine))


Saving images for TCGA-06-0187
Saving images for TCGA-06-0241
Saving images for TCGA-08-0392
